In [ ]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from IPython.display import Markdown
from plotly.express.colors import qualitative as qual_colours
from pathlib import Path
import math
import plotly.express as px
import plotly.graph_objects as go

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_spaghetti
from emu_renewal.outputs import plot_uncertainty_patches, PANEL_SUBTITLES, plot_3d_spaghetti, plot_post_prior_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.utils import get_adjust_idata_index, adjust_summary_cols

from plotting import plot_main
from utils import load_target_data, load_mobility_data, load_vaccination_data, load_variant_prevalence_data

In [ ]:
# Set country
country = 'Philippines'

In [ ]:
# load data
target_data = load_target_data(country)
mobility_data = load_mobility_data(country)
vaccination_data = load_vaccination_data(country)
variant_data = load_variant_prevalence_data(country)

In [ ]:
# Specify fixed parameters and get calibration data
proc_update_freq = 21
init_time = 50
data = target_data
pop = 116e6
analysis_start = datetime(2021, 5, 1)
analysis_end = datetime(2022, 4, 30)
init_start = analysis_start - timedelta(init_time)
init_end = analysis_start - timedelta(1)
select_data = data.loc[analysis_start: analysis_end]
init_data = data.resample("D").asfreq().interpolate().loc[init_start: init_end] / 7.0

In [ ]:
# Define renewal model and proc update fitting method
proc_fitter = CosineMultiCurve()
renew_model = RenewalModel(pop, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), init_time, init_data, GammaDens())

In [ ]:
# Define priors for calibrated parameter
priors = {
    "gen_mean": dist.TruncatedNormal(7.3, 0.5, low=1.0),
    "gen_sd": dist.TruncatedNormal(3.8, 0.5, low=1.0),
    "cdr": dist.Beta(16, 40), #16, 40
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.TruncatedNormal(8, 0.5, low=1.0),
    "report_sd": dist.TruncatedNormal(3, 0.5, low=1.0),
    "prop_immune": dist.Beta(8, 10) #8, 10
}

In [ ]:
# Run calibration - fitting to weekly cases
calib = StandardCalib(renew_model, priors, select_data, indicator='weekly_sum')
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=infer.init_to_uniform(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=4, num_samples=4000, num_warmup=500)
mcmc.run(random.PRNGKey(1))

In [ ]:
idata = az.from_dict(mcmc.get_samples(True))
idata_sampled = az.extract(idata, num_samples=800)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
def get_full_result(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd, prop_immune):
    return renew_model.renewal_func(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd, prop_immune)

full_wrap = jit(get_full_result)
panel_subtitles = ["weekly_sum"] + PANEL_SUBTITLES[1:]
spaghetti = get_spaghetti_from_params(renew_model, sample_params, full_wrap, outputs=panel_subtitles)
quantiles_df = get_quant_df_from_spaghetti(renew_model, spaghetti, quantiles=[0.05, 0.5, 0.95], outputs=panel_subtitles)

In [ ]:
# Restrict range of mobility and vax data to analysis timeframe
mobility_data = mobility_data.loc[analysis_start:analysis_end]
vaccination_data = vaccination_data.loc[analysis_start:analysis_end]

In [ ]:
fig = plot_main(quantiles_df, select_data, mobility_data, vaccination_data).update_layout(showlegend=False)

# Add interventions and variant prevalence to plots
number = 2
for i in range(number):
    
    delta = variant_data[variant_data['variant'] == 'Delta']
    delta_week = delta['week'].to_string(index=False)
    fig.add_vline(x=datetime.strptime(delta_week, "%Y-%m-%d").timestamp() * 1000, 
              annotation_text="Delta >50%", annotation_position="bottom right", row=1, col=i+1, line_dash="dash")
    
    omicron = variant_data[variant_data['variant'] == 'Omicron_BA1_2']
    omicron_week = omicron['week'].to_string(index=False)
    fig.add_vline(x=datetime.strptime(omicron_week, "%Y-%m-%d").timestamp() * 1000, 
              annotation_text="Omicron >50%", annotation_position="bottom right", row=1, col=i+1, line_dash="dash")

fig['layout']['xaxis5']['title']='Date'
fig['layout']['xaxis6']['title']='Date'
fig['layout']['yaxis']['title']='Weekly reported cases'
fig['layout']['yaxis2']['title']=''
fig['layout']['yaxis3']['title']='Total persons susceptible'
fig['layout']['yaxis4']['title']='' 
fig['layout']['yaxis5']['title']='% change from baseline'
fig['layout']['yaxis6']['title']='% total population' 

fig.show()

In [ ]:
# Quick code for exploring quantiles
print(quantiles_df.loc['2021-12-20':'2022-1-10'])

In [ ]:
# quick code for calculating final attack rate
suscepts = quantiles_df['susceptibles']
suscepts['AR_05'] = 1 - suscepts[0.05]/pop
suscepts['AR_5'] = 1 - suscepts[0.50]/pop
suscepts['AR_95'] = 1- suscepts[0.95]/pop

In [ ]:
suscepts.tail(5)

In [ ]:
Markdown(renew_model.get_description())

In [ ]:
Markdown(calib.get_description())

In [ ]:
az.summary(idata)

In [ ]:
plot_post_prior_comparison(idata, list(priors.keys()), priors);

In [ ]:
## second model with different priors

In [ ]:
# Define parameter ranges
priors_2 = {
    "gen_mean": dist.TruncatedNormal(7.3, 0.5, low=1.0),
    "gen_sd": dist.TruncatedNormal(3.8, 0.5, low=1.0),
    "cdr": dist.Beta(4, 10),
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.TruncatedNormal(8, 0.5, low=1.0),
    "report_sd": dist.TruncatedNormal(3, 0.5, low=1.0),
    "prop_immune": dist.Uniform(0.35, 0.55)
}